In [13]:
#Here we will import the two libraries needed. Statsbomb to access the open data and pandas for data analysis.
from statsbombpy import sb
import pandas as pd

euro = sb.matches(competition_id = 55, season_id=43)


credentials were not supplied. open data access only


In [14]:
filt = (euro['home_team'] == 'Italy') | (euro['away_team'] == 'Italy')

ITAgames = euro[filt]

final_match_id = euro.at[4, 'match_id']

final_events = sb.events(match_id=final_match_id, split=True, flatten_attrs=False)

credentials were not supplied. open data access only


In [15]:
final_events['passes']

final_events['passes'].keys()

all_passes = final_events['passes'][['period','location','player','pass']].copy()
all_passes

,period,location,player,pass
0,1,"[60.0, 40.0]",Ciro Immobile,"{'recipient': {'id': 3166, 'name': 'Marco Verr..."
1,1,"[48.0, 35.7]",Marco Verratti,"{'recipient': {'id': 11514, 'name': 'Giovanni ..."
2,1,"[39.2, 68.4]",Giovanni Di Lorenzo,"{'recipient': {'id': 7173, 'name': 'Leonardo B..."
3,1,"[25.3, 46.4]",Leonardo Bonucci,"{'recipient': {'id': 6954, 'name': 'Giorgio Ch..."
4,1,"[23.9, 18.6]",Giorgio Chiellini,"{'recipient': {'id': 7173, 'name': 'Leonardo B..."
...,...,...,...,...
1345,4,"[6.0, 44.0]",Jordan Pickford,"{'recipient': {'id': 10955, 'name': 'Harry Kan..."
1346,4,"[81.2, 52.6]",Harry Kane,"{'length': 27.294687, 'angle': 0.7802169, 'hei..."
1347,4,"[19.5, 8.3]",Alessandro Florenzi,"{'recipient': {'id': 6993, 'name': 'Bryan Cris..."
1348,4,"[96.9, 80.0]",Jack Grealish,"{'recipient': {'id': 22084, 'name': 'Bukayo Sa..."


In [16]:
pass_start = pd.DataFrame(all_passes['location'].to_list(),columns=['x_start','y_start'])
all_passes.join(pass_start)



,period,location,player,pass,x_start,y_start
0,1,"[60.0, 40.0]",Ciro Immobile,"{'recipient': {'id': 3166, 'name': 'Marco Verr...",60.0,40.0
1,1,"[48.0, 35.7]",Marco Verratti,"{'recipient': {'id': 11514, 'name': 'Giovanni ...",48.0,35.7
2,1,"[39.2, 68.4]",Giovanni Di Lorenzo,"{'recipient': {'id': 7173, 'name': 'Leonardo B...",39.2,68.4
3,1,"[25.3, 46.4]",Leonardo Bonucci,"{'recipient': {'id': 6954, 'name': 'Giorgio Ch...",25.3,46.4
4,1,"[23.9, 18.6]",Giorgio Chiellini,"{'recipient': {'id': 7173, 'name': 'Leonardo B...",23.9,18.6
...,...,...,...,...,...,...
1345,4,"[6.0, 44.0]",Jordan Pickford,"{'recipient': {'id': 10955, 'name': 'Harry Kan...",6.0,44.0
1346,4,"[81.2, 52.6]",Harry Kane,"{'length': 27.294687, 'angle': 0.7802169, 'hei...",81.2,52.6
1347,4,"[19.5, 8.3]",Alessandro Florenzi,"{'recipient': {'id': 6993, 'name': 'Bryan Cris...",19.5,8.3
1348,4,"[96.9, 80.0]",Jack Grealish,"{'recipient': {'id': 22084, 'name': 'Bukayo Sa...",96.9,80.0


In [20]:
pass_info = all_passes['pass'].apply(pd.Series)
pass_info
recipient_name = pass_info['recipient'].apply(pd.Series)
recipient_name.rename(columns={'name':'recipient_name'},inplace=True)
pass_end = pd.DataFrame(pass_info['end_location'].to_list(),columns=['x_end','y_end'])

df = pd.concat([all_passes,pass_start,pass_end, recipient_name,pass_info],axis=1)

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
1345   NaN
1346   NaN
1347   NaN
1348   NaN
1349   NaN
Name: 0, Length: 1350, dtype: float64

In [26]:
slim_df = df[['player','x_start','y_start','x_end','y_end']]
slim_df.head()

,player,x_start,y_start,x_end,y_end
0,Ciro Immobile,60.0,40.0,49.5,33.2
1,Marco Verratti,48.0,35.7,39.0,70.7
2,Giovanni Di Lorenzo,39.2,68.4,24.9,46.7
3,Leonardo Bonucci,25.3,46.4,24.4,17.7
4,Giorgio Chiellini,23.9,18.6,16.4,44.1


In [41]:
ita_lineup = sb.lineups(final_match_id)['Italy']
eng_lineup = sb.lineups(final_match_id)['England']

credentials were not supplied. open data access only
credentials were not supplied. open data access only


In [42]:
ita_squad = ita_lineup['player_name'].to_list()
eng_squad = eng_lineup['player_name'].to_list()


In [51]:
# Solved the problem of filtering df by English/Italian team via 
# https://stackoverflow.com/questions/41518920/python-pandas-how-to-query-if-a-list-type-column-contains-something
# slightly changed it with a reverse because I wanted my cell value to be in (or to be contained by) a different list
ita_mask = df.player.apply(lambda x: x in ita_squad)
eng_mask = df.player.apply(lambda x: x in eng_squad)
ita_df = df[ita_mask]
eng_df = df[eng_mask]

In [53]:
eng_df[eng_df['player']=='John Stones']

,period,location,player,pass,x_start,y_start,x_end,y_end,id,recipient_name,...,switch,cross,assisted_shot_id,goal_assist,through_ball,inswinging,shot_assist,no_touch,miscommunication,cut_back
7,1,"[60.7, 67.7]",John Stones,"{'recipient': {'id': 4706, 'name': 'Kalvin Phi...",60.7,67.7,68.2,67.4,4706.0,Kalvin Phillips,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,1,"[41.2, 42.9]",John Stones,"{'recipient': {'id': 3468, 'name': 'Jordan Pic...",41.2,42.9,14.2,39.5,3468.0,Jordan Pickford,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,1,"[25.0, 50.9]",John Stones,"{'recipient': {'id': 3233, 'name': 'Raheem Ste...",25.0,50.9,52.2,58.2,3233.0,Raheem Sterling,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179,1,"[36.6, 29.7]",John Stones,"{'recipient': {'id': 3233, 'name': 'Raheem Ste...",36.6,29.7,55.5,46.2,3233.0,Raheem Sterling,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,1,"[41.8, 51.3]",John Stones,"{'recipient': {'id': 3205, 'name': 'Kyle Walke...",41.8,51.3,49.9,67.9,3205.0,Kyle Walker,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,1,"[9.7, 34.9]",John Stones,"{'recipient': {'id': 3336, 'name': 'Harry Magu...",9.7,34.9,6.3,25.4,3336.0,Harry Maguire,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213,1,"[28.5, 50.4]",John Stones,"{'recipient': {'id': 3336, 'name': 'Harry Magu...",28.5,50.4,32.1,28.7,3336.0,Harry Maguire,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
226,1,"[82.2, 73.6]",John Stones,"{'recipient': {'id': 3308, 'name': 'Kieran Tri...",82.2,73.6,81.7,77.2,3308.0,Kieran Trippier,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
271,1,"[69.7, 37.9]",John Stones,"{'recipient': {'id': 10955, 'name': 'Harry Kan...",69.7,37.9,83.2,30.6,10955.0,Harry Kane,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278,1,"[66.2, 37.8]",John Stones,"{'recipient': {'id': 3336, 'name': 'Harry Magu...",66.2,37.8,66.7,19.4,3336.0,Harry Maguire,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
